In [1]:
import bs4

In [2]:
from bs4 import BeautifulSoup

In [3]:
import requests

In [4]:
dir(requests)

['ConnectTimeout',
 'ConnectionError',
 'DependencyWarning',
 'FileModeWarning',
 'HTTPError',
 'JSONDecodeError',
 'NullHandler',
 'PreparedRequest',
 'ReadTimeout',
 'Request',
 'RequestException',
 'RequestsDependencyWarning',
 'Response',
 'Session',
 'Timeout',
 'TooManyRedirects',
 'URLRequired',
 '__author__',
 '__author_email__',
 '__build__',
 '__builtins__',
 '__cached__',
 '__cake__',
 '__copyright__',
 '__description__',
 '__doc__',
 '__file__',
 '__license__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__title__',
 '__url__',
 '__version__',
 '_check_cryptography',
 '_internal_utils',
 'adapters',
 'api',
 'auth',
 'certs',
 'chardet_version',
 'charset_normalizer_version',
 'check_compatibility',
 'codes',
 'compat',
 'cookies',
 'delete',
 'exceptions',
 'get',
 'head',
 'hooks',
 'logging',
 'models',
 'options',
 'packages',
 'patch',
 'post',
 'put',
 'request',
 'session',
 'sessions',
 'ssl',
 'status_codes',
 'structures',
 'urllib3',
 

In [5]:
url = "https://books.toscrape.com"

In [6]:
session = requests.Session()

In [7]:
try:
    response = session.get(url, timeout=10)
    response.raise_for_status()
    print(f"fetched: {url}")
except requests.exceptions.HTTPError as errh:
    print("HTTP error occurred:\n", errh)
except requests.exceptions.RequestException as err:
    print("Unexpected request error:\n", err)
except Exception as e:
    print(f"Unhandled exception:\n{type(e).__name__}: {e}")

fetched: https://books.toscrape.com


In [8]:
status = response.status_code
status

200

In [9]:
html = response.text
#html

In [10]:
soup = BeautifulSoup(html, "lxml")
#soup

In [11]:
books = soup.select("article.product_pod")
#books

In [12]:
type(books[0])

bs4.element.Tag

In [13]:
books[0]

<article class="product_pod">
<div class="image_container">
<a href="catalogue/a-light-in-the-attic_1000/index.html"><img alt="A Light in the Attic" class="thumbnail" src="media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg"/></a>
</div>
<p class="star-rating Three">
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
</p>
<h3><a href="catalogue/a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a></h3>
<div class="product_price">
<p class="price_color">Â£51.77</p>
<p class="instock availability">
<i class="icon-ok"></i>
    
        In stock
    
</p>
<form>
<button class="btn btn-primary btn-block" data-loading-text="Adding..." type="submit">Add to basket</button>
</form>
</div>
</article>

In [14]:
book1 = books[0]

In [15]:
dir(book1)

['EMPTY_ELEMENT_EVENT',
 'END_ELEMENT_EVENT',
 'MAIN_CONTENT_STRING_TYPES',
 'START_ELEMENT_EVENT',
 'STRING_ELEMENT_EVENT',
 '_TreeTraversalEvent',
 '__annotations__',
 '__bool__',
 '__call__',
 '__class__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__firstlineno__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__static_attributes__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 '__weakref__',
 '_all_strings',
 '_event_stream',
 '_find_all',
 '_find_one',
 '_format_tag',
 '_indent_string',
 '_insert',
 '_is_xml',
 '_lastRecursiveChild',
 '_last_descendant',
 '_namespaces',
 '_self_and',
 '_should_pretty_print'

In [16]:
for child in book1.children:
    print(child)



<div class="image_container">
<a href="catalogue/a-light-in-the-attic_1000/index.html"><img alt="A Light in the Attic" class="thumbnail" src="media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg"/></a>
</div>


<p class="star-rating Three">
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
</p>


<h3><a href="catalogue/a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a></h3>


<div class="product_price">
<p class="price_color">Â£51.77</p>
<p class="instock availability">
<i class="icon-ok"></i>
    
        In stock
    
</p>
<form>
<button class="btn btn-primary btn-block" data-loading-text="Adding..." type="submit">Add to basket</button>
</form>
</div>




In [17]:
for book in books:
    print("image data")
    image = book.select(".image_container")
    print(image)

    print("rating data")
    rating = book.select("star-rating")
    print(rating)

    print("book title")
    title = book.select("h3.a")
    print(title)

    print("price and availability")
    price = book.select("product_price")
    print(price)

    

image data
[<div class="image_container">
<a href="catalogue/a-light-in-the-attic_1000/index.html"><img alt="A Light in the Attic" class="thumbnail" src="media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg"/></a>
</div>]
rating data
[]
book title
[]
price and availability
[]
image data
[<div class="image_container">
<a href="catalogue/tipping-the-velvet_999/index.html"><img alt="Tipping the Velvet" class="thumbnail" src="media/cache/26/0c/260c6ae16bce31c8f8c95daddd9f4a1c.jpg"/></a>
</div>]
rating data
[]
book title
[]
price and availability
[]
image data
[<div class="image_container">
<a href="catalogue/soumission_998/index.html"><img alt="Soumission" class="thumbnail" src="media/cache/3e/ef/3eef99c9d9adef34639f510662022830.jpg"/></a>
</div>]
rating data
[]
book title
[]
price and availability
[]
image data
[<div class="image_container">
<a href="catalogue/sharp-objects_997/index.html"><img alt="Sharp Objects" class="thumbnail" src="media/cache/32/51/3251cf3a3412f53f339e42cac2134093

In [18]:
##### image data

In [19]:
image = book1.select_one(".image_container")
image

<div class="image_container">
<a href="catalogue/a-light-in-the-attic_1000/index.html"><img alt="A Light in the Attic" class="thumbnail" src="media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg"/></a>
</div>

In [20]:
book1.select_one(".image_container a").get("href")
image.select_one("a").get("href")
image.a.get("href")

'catalogue/a-light-in-the-attic_1000/index.html'

In [21]:
img = book1.select_one(".image_container img")
img = image.select_one("img")
img = image.img

In [22]:
img.get("alt")

'A Light in the Attic'

In [23]:
img.get("src")

'media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg'

In [24]:
# rating data
rating = book1.select_one(".star-rating")
rating

<p class="star-rating Three">
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
</p>

##### book title

In [25]:
title = book1.select_one("h3 a")
title = book1.h3.a
title

<a href="catalogue/a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a>

In [26]:
title.get("href")

'catalogue/a-light-in-the-attic_1000/index.html'

In [27]:
title.get("title")

'A Light in the Attic'

##### price and availability

In [32]:
price = book1.select_one(".product_price")
price

<div class="product_price">
<p class="price_color">Â£51.77</p>
<p class="instock availability">
<i class="icon-ok"></i>
    
        In stock
    
</p>
<form>
<button class="btn btn-primary btn-block" data-loading-text="Adding..." type="submit">Add to basket</button>
</form>
</div>

In [36]:
price.select_one(".price_color").text

'Â£51.77'

In [39]:
price.select_one(".availability").get_text(strip=True)

'In stock'